In [ ]:
!pip install transformers datasets evaluate accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv("/content/data/train.csv")
val_df = pd.read_csv("/content/data/validation.csv")
test_df = pd.read_csv("/content/data/test.csv")

In [ ]:
#Cleaning Data
def clean_data(df):
  df = df.copy()
  df = df.dropna(subset=["sentence", "sentiment"])
  df["sentence"] = df["sentence"].astype(str)
  df["sentiment"] = df["sentiment"].astype(int)
  return df

train_df = clean_data(train_df)
test_df = clean_data(test_df)
val_df = clean_data(val_df)

In [ ]:
#Create DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

In [ ]:
dataset["train"][0]

{'sentence': 'slide giáo trình đầy đủ .', 'sentiment': 2, 'topic': 1}

In [ ]:
model_checkpoint = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
#Standardization
def tokenize_func(examples):
  return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_func, batched=True)

Map:   0%|          | 0/11426 [00:00<?, ? examples/s]

Map:   0%|          | 0/1583 [00:00<?, ? examples/s]

Map:   0%|          | 0/3166 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3166
    })
})


In [ ]:
#Need to rename column into labels - the Trainer automatically find it
tokenized_dataset = tokenized_dataset.rename_column("sentiment", "labels")
tokenized_dataset = tokenized_dataset.remove_columns(["sentence", "topic"])
# Delete index created by pandas
for split in tokenized_dataset.keys():
    if "__index_level_0__" in tokenized_dataset[split].column_names:
        tokenized_dataset[split] = tokenized_dataset[split].remove_columns(["__index_level_0__"])

In [ ]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3166
    })
})


In [ ]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1") #For imbalanced dataset

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    return {**acc, **f1} # ** ~ unpack the dictionary

In [ ]:
#Labeling
id2label = {0: "Tiêu cực", 1: "Trung tính", 2: "Tích cực"}
label2id = {"Tiêu cực": 0, "Trung tính": 1, "Tích cực": 2}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="phobert_student_feedback",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3966522872.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bachnguyen0322 (bachnguyen0322-ming-chuan-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.305000,0.228455,0.941251,0.938906
2,0.211000,0.235098,0.940619,0.938278
3,0.125300,0.258181,0.944409,0.941119


TrainOutput(global_step=2145, training_loss=0.1964077627186453, metrics={'train_runtime': 963.0138, 'train_samples_per_second': 35.595, 'train_steps_per_second': 2.227, 'total_flos': 2254750433220096.0, 'train_loss': 0.1964077627186453, 'epoch': 3.0})

In [ ]:
test_results = trainer.evaluate(tokenized_dataset["test"])
print(test_results)

{'eval_loss': 0.2669762074947357, 'eval_accuracy': 0.9289324068224889, 'eval_f1': 0.9267411016871413, 'eval_runtime': 26.9962, 'eval_samples_per_second': 117.276, 'eval_steps_per_second': 7.334, 'epoch': 3.0}


In [ ]:
save_path = "phobert_student_feedback_final"

trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

('phobert_student_feedback_final/tokenizer_config.json',
 'phobert_student_feedback_final/special_tokens_map.json',
 'phobert_student_feedback_final/vocab.txt',
 'phobert_student_feedback_final/bpe.codes',
 'phobert_student_feedback_final/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load model
classifier = pipeline("text-classification", model=save_path, tokenizer=save_path)

# Test
examples = [
    "Thầy dạy nhiệt tình, bài giảng hay.",
    "Phòng học nóng quá, slide chữ bé tí không nhìn thấy gì.",
    "Môn học bình thường, không có gì đặc sắc."
]

results = classifier(examples)

for text, res in zip(examples, results):
    print(f"Câu: {text}")
    print(f"Dự đoán: {res['label']} (Độ tin cậy: {res['score']:.2f})")
    print("-" * 30)

Device set to use cuda:0


Câu: Thầy dạy nhiệt tình, bài giảng hay.
Dự đoán: Tích cực (Độ tin cậy: 1.00)
------------------------------
Câu: Phòng học nóng quá, slide chữ bé tí không nhìn thấy gì.
Dự đoán: Tiêu cực (Độ tin cậy: 0.99)
------------------------------
Câu: Môn học bình thường, không có gì đặc sắc.
Dự đoán: Tiêu cực (Độ tin cậy: 0.82)
------------------------------


In [ ]:
import shutil
from google.colab import files

# Nén folder thành file zip
shutil.make_archive("my_model", 'zip', "phobert_student_feedback_final")

# Tải xuống
files.download("my_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>